In [1]:
import datetime
from dateutil import parser
import math
from dateutil.parser import parse
from geopandas import GeoDataFrame
import pandas as pd
import numpy as np
from shapely.geometry import Point
import json
import ast
import sys
import pickle
from __future__ import print_function
import os
import time
import collections

import glob
import json
import re
import pandas as pd
import sys
import pickle
import numpy as np
import copy
import math
import ipdb
import matplotlib.pyplot as plt
import ast


folder_project = "/Users/sergiocamelo/Dropbox/Sergio-Joann/"
os.chdir(folder_project)

sys.path.insert(0, 'Code/VRPEngine/pyCode')
sys.path.insert(0, 'Code/VRPEngine/C++Engine')
sys.path.insert(0, 'Code/VRPEngine/pyCode/tsp')

import solver as solver
import distances as distances
import VRPClass
import road_distance


In [2]:
days_analysis = 14
ts = 'Jan242019-nopenalty'
data_folder = 'StandardizedData/'
results_folder_ts = 'Results/'+ ts + "/" 
folder_instances = results_folder_ts+"instances"
solution_folder = results_folder_ts+'solution_routes/'
dim_farmers_file = 'dim_farmers_sample.csv'
dim_all_farmers_file = 'dim_all_farmers.csv'
dim_confirmed_farmers_file = 'dim_farmers_confirmed_old.csv'



In [3]:
truck_loads = []

In [4]:
# Get coordinates of farmers, mills and trucks
dim_pickups = pd.read_csv(results_folder_ts+'data_cleaning_results/dim_pickups.csv')

dim_trucks = pd.read_csv(results_folder_ts+'data_cleaning_results/dim_trucks.csv')
dim_farmers = pd.read_csv(data_folder+dim_farmers_file)
dim_all_farmers = pd.read_csv(data_folder+dim_all_farmers_file)
dim_confirmed_farmers = pd.read_csv(data_folder+dim_confirmed_farmers_file)


dim_mills = pd.read_csv(data_folder+'dim_mills.csv')
dim_middlemen = pd.read_csv(data_folder+'dim_middlemen.csv')

positions_dict = {}
for i, row in dim_mills.iterrows():
    positions_dict[row["code"]] = (row['latitude'], row['longitude'])
for i, row in dim_all_farmers.iterrows():
    positions_dict[row["farmer_id"]+'-'+str(row["plot_number"])] = (row['latitude'], row['longitude'])
for i, row in dim_confirmed_farmers.iterrows():
    positions_dict[row["farmer_id"]+'-'+str(row["plot_number"])] = (row['latitude'], row['longitude'])
for i, row in dim_farmers.iterrows():
    positions_dict[row["farmer_id"]+'-'+str(row["plot_number"])] = (row['latitude'], row['longitude'])
for i, row in dim_middlemen.iterrows():
    positions_dict[row["cluster_id"]] = (row['latitude'], row['longitude'])
for i, row in dim_trucks.iterrows():
    positions_dict[row["truck_id"]] = positions_dict[float(row["cluster_id"])]
    
def get_distance(a, b, mapping = None, detailed = True):
    if a[0] == 'h':
        m_a = mapping[a][0]
    elif a[0] == 'n':
        m_a = mapping[a]
    else:
        m_a = a
    if b[0] == 'h':
        m_b = mapping[b][0]
    elif b[0] == 'n':
        m_b = mapping[b]
    else:
        m_b = b
    return road_distance.road_distance(positions_dict[m_a],positions_dict[m_b], detailed=detailed)

def path_distance(path, mapping, mill):
    surfaces = ['access','gravel','asphalt','dirt','unknown']
    new_path = copy.copy(path)
    new_path.insert(len(path)-1, mill)
    result = None
    for i in range(len(new_path)-1):
        if not result:
            result = get_distance(new_path[i],new_path[i+1],mapping)
            #ipdb.set_trace()
        else:
            dist = get_distance(new_path[i],new_path[i+1],mapping)

            result["distance"] += dist["distance"]
            result["time"] += dist["time"]
            for k in dist["detailed_distance"].keys():
                if k in result["detailed_distance"].keys():
                    result["detailed_distance"][k]+=dist["detailed_distance"][k]
                    result["detailed_time"][k]+=dist["detailed_time"][k]
                else:
                    result["detailed_distance"][k]=dist["detailed_distance"][k]
                    result["detailed_time"][k]=dist["detailed_time"][k]
    
    for s in surfaces:
        if not(s in result["detailed_distance"].keys()):
            result["detailed_distance"][s] = 0
            result["detailed_time"][s] = 0
    
    return result
        
        

In [5]:
def clean_dict(d):
    d = ast.literal_eval(d)
    for k in d.keys():
        if d[k] == 0:
            del d[k]
    return d
dim_middlemen = dim_middlemen[pd.notnull(dim_middlemen['cluster_id'])]
dim_middlemen['cluster_id'] = dim_middlemen['cluster_id'].apply(int).apply(str)
dim_middlemen['trucks'] = dim_middlemen['trucks'].apply(clean_dict)

In [6]:
# Remove repeated farmers
def remove_repetitions(data,N,vrp):
    
    # Recalculate distances and verify that they are the same
    new_ub = 0
    for t in data['routes'].keys():
        for i,route in enumerate(data['routes'][t]):
            route["distance"] = vrp.distance_path([t]+route['route']+[t])
            new_ub += route["distance"]
    if (np.abs(new_ub-data['z_ub']) > 0.1):
        print("Error in distance calculation", new_ub,data['z_ub'])
    
    # Check which farmers are picked up
    ocurrences = dict(zip(N,[0]*len(N)))
    for t in data['routes'].keys():
        for route in data['routes'][t]:
            for n in route['route']:
                ocurrences[n]+=1

    for n in N:
        if ocurrences[n]>1:
            
            print("Node %s is repeated" % n)
            # Calculate gain from removing it
            min_gain = float('inf')
            min_route = None
            min_coord = None
            for t in data['routes'].keys():
                for i,route in enumerate(data['routes'][t]):
                    if n in route['route']:
                        route_with = route['route']
                        route_without = [m for m in route['route'] if m!=n]
                        dist_before = vrp.distance_path([t]+route_with+[t])
                        dist_after = vrp.distance_path([t]+route_without+[t])
                        gain = dist_before - dist_after
                        if gain <= min_gain:
                            min_gain = gain
                            min_route = copy.copy(route_with)
                            min_coord = (t,i)
                        route['route'] = route_without
            data['routes'][min_coord[0]][min_coord[1]]['route'] = min_route
            
            for t in data['routes'].keys():
                for i,route in enumerate(data['routes'][t]):
                    route["distance"] = vrp.distance_path([t]+route['route']+[t])
                    new_ub += route["distance"]
            data['z_ub'] = new_ub  
        if ocurrences[n]<1:
            print("Error, there is a node without pickup route")


    ocurrences = dict(zip(N,[0]*len(N)))
    for t in data['routes'].keys():
        for route in data['routes'][t]:
            for n in route['route']:
                ocurrences[n]+=1

    for n in N:
        if ocurrences[n]>1:
            print("Error, fixing not successful")
        if ocurrences[n]<1:
            print("Error, fixing not successful")
            
            
    # Recalculate Upper Bound
    old_ub = data['z_ub']
    new_ub = 0
    for t in data['routes'].keys():
        for i,route in enumerate(data['routes'][t]):
            route["distance"] = vrp.distance_path([t]+route['route']+[t])
            new_ub += route["distance"]
    data['z_ub'] = new_ub  
    if old_ub < new_ub - 0.1:
        print("Error in new upper bound")
        print(old_ub, new_ub)
    try:
        gap = (data['z_ub']-data['z_lb'])/data['z_ub']
        if gap < -10**(-8):
            print("Error, negative gap")
            print(data['z_ub'])
            print(data['z_lb'])
    except:
        print("zero z_ub")

In [7]:
# Create a dictionary of farmer to original middleman, to check whether they are retained or not
original_middleman = dict(zip(dim_farmers['farmer_id']+'-'+dim_farmers['plot_number'].apply(str),dim_farmers['cluster_id']))
for i,row in dim_pickups.iterrows():
    original_middleman[row['plot_id']] = row['cluster_id']

In [8]:
# Load daily results
instances = glob.glob(solution_folder+"/daily/*.json")
print("There is a total of %d daily instances"%len(instances))
daily_results = {}
instances_parsed = []

z_ub = 0
z_lb = 0
for instance in instances:
    # Load instance
    with open(instance) as f:
        data = json.load(f)
    name_instance = re.search('/daily/(.*).json',instance).group(1)

    # Get day and cluster
    day = name_instance.split('_')[4]
    cluster = name_instance.split('_')[2]
    
    # Get the data of this instance
    instance_data = pickle.load( open( folder_instances+"/daily/daily_cluster_"+str(cluster)+"_day_"+str(day) + ".p", "rb" ) )
    locals().update(instance_data)
    vrp = VRPClass.VRP(H, N, H_p, N_p, quantities, capacities, type_dist, M = M, M_p = M_p, distance_matrix = distance)
    data['mapping'] = mapping

    # Get farmers served
    farmers = [node for node in data['mapping'].keys() if node[0] == 'n']
    farmers_index = [data['mapping'][f] for f in farmers]
    # Make sure these farmers are part of the routes
    farmers_inroute = [n for node in [route['route'] for key, routes in data['routes'].iteritems() if len(routes)>0 for route in routes] for n in node]
    if (set(farmers_inroute)!=set(farmers)):
        print("error")
        
    # Fix farmers that are picked up more than once
    remove_repetitions(data,N,vrp)
    
    instances_parsed.append({
        "n_nodes": len([n for n in data['mapping'].keys() if n[0]=='n']),
        "cluster": cluster,
        "day": day,
        "n_trucks": len([n for n in data['mapping'].keys() if n[0]=='h']),
        "z_ub":data['z_ub'],
        "z_lb":data['z_lb'],
        "gap":(data['z_ub']-data['z_lb'])/data['z_ub'],
        "nodes_served":[data['mapping'][n] for n in data['mapping'].keys() if n[0]=='n']
    })
    
    for t in data['routes'].keys():
        for route in data['routes'][t]:
            detailed_dist = path_distance([t]+route['route']+[t],mapping,'SKIP')
            # The route with the real ids
            real_route = [data['mapping'][t][0]]+[data['mapping'][n] for n in route["route"] if n[0]=='n']+['SKIP']+[data['mapping'][t][0]]
            truck_loads.append({
                    "type":"daily",
                    "cluster":data['mapping'][t][2],
                    "truck":data['mapping'][t][0],
                    "day":data['mapping'][t][1],
                    "max_load":route["max_load"],
                    "load":np.sum([quantities[n] for n in route['route']]),
                    "loads":[quantities[n] for n in route['route']],
                    "load_perc":np.sum([quantities[n] for n in route['route']])/(route["max_load"]+0.0),
                    "n_farmers":len(set(route["route"])),
                    "distance":vrp.distance_path([t]+route['route']+[t]),
                    "detailed_distance":detailed_dist['distance'],
                    "gravel":detailed_dist['detailed_distance']['gravel'],
                    "asphalt":detailed_dist['detailed_distance']['asphalt'],
                    "access":detailed_dist['detailed_distance']['access'],
                    "dirt":detailed_dist['detailed_distance']['dirt'],
                    "unknown":detailed_dist['detailed_distance']['unknown'],
                    "road_time":detailed_dist['time'],
                    "farmers":[data['mapping'][n] for n in route["route"] if n[0]=='n'],
                    "route":real_route,
                    # This contains the node and its location
                    "waypoints":[(r,positions_dict[r]) for r in real_route],
                    "retained_farmer":[int(i) for i in np.array([original_middleman[data['mapping'][n]] for n in route["route"] if n[0]=='n'] ) == data['mapping'][t][2]],
                    "retained_farmers":np.sum(np.array([original_middleman[data['mapping'][n]] for n in route["route"] if n[0]=='n'] ) == data['mapping'][t][2])
                })
    #print(instance)
    #print((data['z_ub']-data['z_lb'])/data['z_ub'])

    
    # Get upper and lower bounds
    z_ub += data['z_ub']
    z_lb += data['z_lb']
print (z_ub)
print (z_lb)
print((z_ub-z_lb)/z_ub)
daily_df = pd.DataFrame(instances_parsed)
daily_df.to_csv(results_folder_ts+("tables_for_report/runs_daily.csv"),index=False)

There is a total of 259 daily instances
Node n_1 is repeated
Node n_2 is repeated
Node n_1 is repeated
Node n_4 is repeated
Node n_2 is repeated
Node n_2 is repeated
Node n_2 is repeated
11260067.283138001
10935414.9662
0.02883218268533994


In [9]:
# Load spatial results
instances = glob.glob(solution_folder+"spatial/*.json")
print("There is a total of %d spatial instances"%len(instances))
daily_results = {}
instances_parsed = []


z_ub = 0
z_lb = 0

for instance in instances:
    print(instance)
    # Load instance
    with open(instance) as fi:
        data = json.load(fi)
    name_instance = re.search('/spatial/(.*).json',instance).group(1)

    # Get day and cluster
    day = name_instance.split('_')[2]
    
    # Get the data of this instance
    instance_data = pickle.load( open( folder_instances+"/spatial/spatial_day_"+str(day) + ".p", "rb" ) )
    locals().update(instance_data)
    vrp = VRPClass.VRP(H, N, H_p, N_p, quantities, capacities, type_dist, M = M, M_p = M_p, distance_matrix = distance)


    # Get farmers served
    farmers = [node for node in data['mapping'].keys() if node[0] == 'n']
    farmers_index = [data['mapping'][f] for f in farmers]
    # Make sure these farmers are part of the routes
    farmers_inroute = [n for node in [route['route'] for key, routes in data['routes'].iteritems() if len(routes)>0 for route in routes] for n in node]
    if (set(farmers_inroute)!=set(farmers)):
        print("error")
        print(farmers_inroute)
        print(farmers)
        print(name_instance)
        print (data['z_lb'])
        print (data['z_ub'])
        
    # Fix farmers that are picked up more than once
    remove_repetitions(data,N,vrp)
    
    # Get upper and lower bounds
    z_ub += data['z_ub']
    z_lb += data['z_lb']
    
    try:
        instances_parsed.append({
            "n_nodes": len([n for n in data['mapping'].keys() if n[0]=='n']),
            "day": day,
            "n_trucks": len([n for n in data['mapping'].keys() if n[0]=='h']),
            "z_ub":data['z_ub'],
            "z_lb":data['z_lb'],
            "gap":(data['z_ub']-data['z_lb'])/data['z_ub'],
            "nodes_served":[data['mapping'][n] for n in data['mapping'].keys() if n[0]=='n']
        })
    except:
        print("zero z_ub")
    
    for t in data['routes'].keys():
        for route in data['routes'][t]:
            detailed_dist = path_distance([t]+route['route']+[t],mapping,'SKIP')
            real_route = [data['mapping'][t][0]]+[data['mapping'][n] for n in route["route"] if n[0]=='n']+['SKIP']+[data['mapping'][t][0]]
            truck_loads.append({
                    "type":"spatial",
                    "cluster":data['mapping'][t][2],
                    "truck":data['mapping'][t][0],
                    "day":data['mapping'][t][1],
                    "max_load":route["max_load"],
                    "load":np.sum([quantities[n] for n in route['route']]),
                    "loads":[quantities[n] for n in route['route']],
                    "load_perc":np.sum([quantities[n] for n in route['route']])/(route["max_load"]+0.0),
                    "n_farmers":len(set(route["route"])),
                    "distance":vrp.distance_path([t]+route['route']+[t]),
                    "detailed_distance":detailed_dist['distance'],
                    "gravel":detailed_dist['detailed_distance']['gravel'],
                    "asphalt":detailed_dist['detailed_distance']['asphalt'],
                    "access":detailed_dist['detailed_distance']['access'],
                    "dirt":detailed_dist['detailed_distance']['dirt'],
                    "unknown":detailed_dist['detailed_distance']['unknown'],
                    "road_time":detailed_dist['time'],
                    "farmers":[data['mapping'][n] for n in route["route"] if n[0]=='n'],
                    "route":real_route,
                    # This contains the node and its location
                    "waypoints":[(r,positions_dict[r]) for r in real_route],
                    "retained_farmer":[int(i) for i in np.array([original_middleman[data['mapping'][n]] for n in route["route"] if n[0]=='n'] ) == data['mapping'][t][2]],
                    "retained_farmers":np.sum(np.array([original_middleman[data['mapping'][n]] for n in route["route"] if n[0]=='n'] ) == data['mapping'][t][2])
            })
print (z_ub)
print (z_lb)
print((z_ub-z_lb)/z_ub)
spatial_df = pd.DataFrame(instances_parsed)

spatial_df.to_csv(results_folder_ts+("tables_for_report/runs_spatial.csv"),index=False)

There is a total of 13 spatial instances
Results/Jan242019-nopenalty/solution_routes/spatial/spatial_day_0.json
Node n_1 is repeated
Node n_5 is repeated
Node n_7 is repeated
Results/Jan242019-nopenalty/solution_routes/spatial/spatial_day_1.json
Results/Jan242019-nopenalty/solution_routes/spatial/spatial_day_10.json
Node n_19 is repeated
Results/Jan242019-nopenalty/solution_routes/spatial/spatial_day_11.json
Node n_0 is repeated
Node n_1 is repeated
Results/Jan242019-nopenalty/solution_routes/spatial/spatial_day_12.json
Node n_9 is repeated
Results/Jan242019-nopenalty/solution_routes/spatial/spatial_day_2.json
Node n_0 is repeated
Node n_23 is repeated
Node n_31 is repeated
Node n_35 is repeated
Node n_42 is repeated
Results/Jan242019-nopenalty/solution_routes/spatial/spatial_day_3.json
Results/Jan242019-nopenalty/solution_routes/spatial/spatial_day_4.json
Results/Jan242019-nopenalty/solution_routes/spatial/spatial_day_5.json
Node n_35 is repeated
Results/Jan242019-nopenalty/solution_r

In [168]:
# # Load temporal results
# instances = glob.glob(solution_folder+"temporal/*.json")
# print("There is a total of %d daily instances"%len(instances))
# daily_results = {}
# instances_parsed = []


# z_ub = 0
# z_lb = 0
# for instance in instances:
#     print(instance)
#     # Load instance
#     with open(instance) as f:
#         data = json.load(f)
#     name_instance = re.search('/temporal/(.*).json',instance).group(1)

#     # Get day and cluster
#     cluster = name_instance.split('_')[2]
    
#     # Get the data of this instance
#     instance_data = pickle.load( open( folder_instances+"/temporal/temporal_cluster_"+str(cluster) + ".p", "rb" ) )
#     locals().update(instance_data)
#     vrp = VRPClass.VRP(H, N, H_p, N_p, quantities, capacities, type_dist, M = M, M_p = M_p, distance_matrix = distance)


#     # Get farmers served
#     farmers = [node for node in data['mapping'].keys() if node[0] == 'n']
#     farmers_index = [data['mapping'][f] for f in farmers]
#     # Make sure these farmers are part of the routes
#     farmers_inroute = [n for node in [route['route'] for key, routes in data['routes'].iteritems() if len(routes)>0 for route in routes] for n in node]
#     if (set(farmers_inroute)!=set(farmers)):
#         print("error")
#         print(farmers_inroute)
#         print(farmers)
#         print(name_instance)
        
#     # Get upper and lower bounds
#     z_ub += data['z_ub']
#     z_lb += data['z_lb']
    
#     # Fix farmers that are picked up more than once
#     remove_repetitions(data,N,vrp)
    
#     instances_parsed.append({
#         "n_nodes": len([n for n in data['mapping'].keys() if n[0]=='n']),
#         "cluster": cluster,
#         "n_trucks": len([n for n in data['mapping'].keys() if n[0]=='h']),
#         "z_ub":data['z_ub'],
#         "z_lb":data['z_lb'],
#         "gap":(data['z_ub']-data['z_lb'])/data['z_ub'],
#         "nodes_served":[data['mapping'][n] for n in data['mapping'].keys() if n[0]=='n']
#     })
    
#     for t in data['routes'].keys():
#         for route in data['routes'][t]:
#             detailed_dist = path_distance([t]+route['route']+[t],mapping,'SKIP')
#             real_route = [data['mapping'][t][0]]+[data['mapping'][n] for n in route["route"] if n[0]=='n']+['SKIP']+[data['mapping'][t][0]]
#             truck_loads.append({
#                     "type":"temporal",
#                     "cluster":int(cluster),
#                     "truck":data['mapping'][t][0],
#                     "day":-1,
#                     "max_load":route["max_load"],
#                     "load":np.sum([quantities[n] for n in route['route']]),
#                     "load_perc":np.sum([quantities[n] for n in route['route']])/(route["max_load"]+0.0),
#                     "n_farmers":len(set(route["route"])),
#                     "distance":vrp.distance_path([t]+route['route']+[t]),
#                     "detailed_distance":detailed_dist['distance'],
#                     "gravel":detailed_dist['detailed_distance']['gravel'],
#                     "asphalt":detailed_dist['detailed_distance']['asphalt'],
#                     "access":detailed_dist['detailed_distance']['access'],
#                     "dirt":detailed_dist['detailed_distance']['dirt'],
#                     "unknown":detailed_dist['detailed_distance']['unknown'],
#                     "road_time":detailed_dist['time'],
#                     "farmers":[data['mapping'][n] for n in route["route"] if n[0]=='n'],
#                     "route":real_route,
#                     # This contains the node and its location
#                     "waypoints":[(r,positions_dict[r]) for r in real_route],
#                     "retained_farmers":np.sum(np.array([original_middleman[data['mapping'][n]] for n in route["route"] if n[0]=='n'] ) == int(cluster))
#             })

# print (z_ub)
# print (z_lb)
# print((z_ub-z_lb)/z_ub)
# temporal_df = pd.DataFrame(instances_parsed)
# temporal_df.to_csv(results_folder+("tables_for_report/runs_temporal.csv"),index=False)

In [9]:
# Extract polygons
for r in truck_loads:
    r["polygon"] = []
    for i in range(len(r["waypoints"])-1):
        r["polygon"] = r["polygon"] + [road_distance.waypoints(r["waypoints"][i][1],r["waypoints"][i+1][1])]


In [10]:
results_folder_ts = 'Results/'+ ts + "/" 

# Save polygons in a JSON file
with open(results_folder_ts + 'parsed_routes.json', 'w') as fp:
    json.dump(truck_loads,fp, sort_keys=True, indent=4)


In [11]:
results_folder_ts

'Results/Jan242019-nopenalty/'

In [12]:
truck_loads

[{'access': 315.865448,
  'asphalt': 15880.8,
  'cluster': 108,
  'day': 0,
  'detailed_distance': 41095.965448,
  'dirt': 11062.9,
  'distance': 41095.965448,
  'farmers': ['F14020081849-1', 'D00000000278-1'],
  'gravel': 10288.8,
  'load': 8.8,
  'load_perc': 0.09777777777777778,
  'loads': [7.0, 1.8],
  'max_load': 90,
  'n_farmers': 2,
  'retained_farmer': [1, 1],
  'retained_farmers': 2,
  'road_time': 9599.462106702125,
  'route': ['t_0', 'F14020081849-1', 'D00000000278-1', 'SKIP', 't_0'],
  'truck': 't_0',
  'type': 'daily',
  'unknown': 0,
  'waypoints': [('t_0', (-0.65444737, 102.57448149999999)),
   ('F14020081849-1', (-0.650714, 102.555748)),
   ('D00000000278-1', (-0.65630605, 102.5697181)),
   ('SKIP', (-0.68269, 102.5015)),
   ('t_0', (-0.65444737, 102.57448149999999))]},
 {'access': 1508.7576139999999,
  'asphalt': 15880.8,
  'cluster': 108,
  'day': 0,
  'detailed_distance': 38017.557614,
  'dirt': 6791.5,
  'distance': 38017.557614,
  'farmers': ['F14020080315-1', 'D00

In [13]:
#Remove day 13
truck_loads = [route for route in truck_loads if route['day']!=13]